In [1]:
import os
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base

# 第一个Agent
- 注意当前使用的langchain版本为0.1.4，调用方式和之前旧版本有所差异

In [39]:

from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import load_tools
from langchain.agents import create_openai_functions_agent #不同的agent有不同的创建方式
from langchain.agents import AgentExecutor
#创建LLM 
llm = ChatOpenAI(model_name="gpt-4", temperature=0)
#定义agent的prompt
#https://smith.langchain.com/hub/hwchase17/openai-functions-agent
prompt = hub.pull("hwchase17/openai-functions-agent")
#定义工具,加载预制的工具,注意有的工具需要提供LLM
tools = load_tools(["llm-math"], llm=llm)
#创建agent
agent = create_openai_functions_agent(llm, tools, prompt)
#定义agent的执行器，这里注意与老版本的不同
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "你好"})



> Entering new AgentExecutor chain...
你好！有什么我可以帮助你的吗？

> Finished chain.


{'input': '你好', 'output': '你好！有什么我可以帮助你的吗？'}

# Agent
- 中间步骤处理
- 提示词
- 模型配置(停止符必要的话)
- 输出解析器
<hr>

In [49]:
from langchain import hub
from langchain.agents import AgentExecutor, tool
from langchain.agents.output_parsers import XMLAgentOutputParser
from langchain_openai import ChatOpenAI

In [57]:
#配置模型
model = ChatOpenAI(
    model="gpt-4-1106-preview",
    temperature=0,
    )
#可用工具
@tool
def search(query: str) -> str:
    """当需要了解最新的天气信息的时候才会使用这个工具。"""
    return "晴朗,32摄氏度,无风"
tool_list = [search]
tool_list

[StructuredTool(name='search', description='search(query: str) -> str - 当需要了解最新的天气信息的时候才会使用这个工具。', args_schema=<class 'pydantic.v1.main.searchSchemaSchema'>, func=<function search at 0x15a7b9800>)]

In [58]:
#提示词模版
# https://smith.langchain.com/hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/xml-agent-convo")
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input', 'tools'], partial_variables={'chat_history': ''}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input', 'tools'], template="You are a helpful assistant. Help the user answer any questions.\n\nYou have access to the following tools:\n\n{tools}\n\nIn order to use a tool, you can use <tool></tool> and <tool_input></tool_input> tags. You will then get back a response in the form <observation></observation>\nFor example, if you have a tool called 'search' that could run a google search, in order to search for the weather in SF you would respond:\n\n<tool>search</tool><tool_input>weather in SF</tool_input>\n<observation>64 degrees</observation>\n\nWhen you are done, respond with a final answer between <final_answer></final_answer>. For example:\n\n<final_answer>The weather in SF is 64 degrees</final_answer>\n\nBegin!\n\nPrevious Conversation:\n{chat_history}\n\n

中间步骤，实现一个log

In [59]:
def convert_intermediate_steps(intermediate_steps):
    log = ""
    for action, observation in intermediate_steps:
        log += (
            f"<tool>{action.tool}</tool><tool_input>{action.tool_input}"
            f"</tool_input><observation>{observation}</observation>"
        )
    return log

In [60]:
#将工具列表插入到模版中
def convert_tools(tools):
    return "\n".join([f"{tool.name}: {tool.description}" for tool in tools])

定义agent

In [61]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: convert_intermediate_steps(
            x["intermediate_steps"]
        ),
    }
    | prompt.partial(tools=convert_tools(tool_list))
    | model.bind(stop=["</tool_input>", "</final_answer>"])
    | XMLAgentOutputParser()
)

In [46]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_function_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history',

In [62]:
#执行agent
agent_executor = AgentExecutor(agent=agent, tools=tool_list, verbose=True)

In [63]:
agent_executor.invoke({"input": "北京今天的天气如何?"})



> Entering new AgentExecutor chain...
<tool>search</tool><tool_input>北京今天的天气晴朗,32摄氏度,无风<final_answer>北京今天的天气是晴朗，温度为32摄氏度，无风。

> Finished chain.


{'input': '北京今天的天气如何?', 'output': '北京今天的天气是晴朗，温度为32摄氏度，无风。'}

## agentType更新版
![agtype.png](agtype.png)

一个比较复杂的agent
- 工具
- 检索增强RAG
- 记忆


In [64]:
#定义工具
import os

os.environ["SERPAPI_API_KEY"] = "f265b8d9834ed7692cba6db6618e2a8a9b24ed6964c457296a2626026e8ed594"
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_community.utilities import SerpAPIWrapper

In [65]:
@tool
def search(query: str) -> str:
    """当需要查找实时信息的时候才会使用这个工具."""
    serp = SerpAPIWrapper()
    return serp.run(query)

print(search.name)
print(search.description)
print(search.args)

search
search(query: str) -> str - 当需要查找实时信息的时候才会使用这个工具.
{'query': {'title': 'Query', 'type': 'string'}}


In [66]:
search("北京今天的天气如何?")

"{'type': 'weather_result', 'temperature': '53', 'unit': 'Fahrenheit', 'precipitation': '0%', 'humidity': '53%', 'wind': '5 mph', 'location': '中国北京市', 'date': 'Saturday', 'weather': 'Partly cloudy'}"

In [68]:
#RAG增强生成
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()
print(f"Loaded {len(docs)} documents")
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

Loaded 1 documents


In [69]:
#搜索匹配文档块
retriever.get_relevant_documents("如何debug?")[0]

Document(page_content='The ability to rapidly understand how the model is performing ‚Äî and debug where it is failing ‚Äî is incredibly important for this phase.Debugging‚ÄãWhen developing new LLM applications, we suggest having LangSmith tracing enabled by default.\nOftentimes, it isn‚Äôt necessary to look at every single trace. However, when things go wrong (an unexpected end result, infinite agent loop, slower than expected execution, higher than expected token usage), it‚Äôs extremely helpful to debug by looking through the application traces. LangSmith gives clear visibility and debugging information at each step of an LLM sequence, making it much easier to identify and root-cause issues.', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows L

In [70]:
#把检索器加入到工具中
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "搜索有关 LangSmith 的信息。关于LangSmith的任何问题，你一定要使用这个工具！",
)
retriever_tool

Tool(name='langsmith_search', description='搜索有关 LangSmith 的信息。关于LangSmith的任何问题，你一定要使用这个工具！', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x14e139da0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x108502cd0>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x14e13a0c0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x108502cd0>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'))

In [72]:
#可用工具集
tools = [search, retriever_tool]
tools

[StructuredTool(name='search', description='search(query: str) -> str - 当需要查找实时信息的时候才会使用这个工具.', args_schema=<class 'pydantic.v1.main.searchSchemaSchema'>, func=<function search at 0x14e9adbc0>),
 Tool(name='langsmith_search', description='搜索有关 LangSmith 的信息。关于LangSmith的任何问题，你一定要使用这个工具！', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x14e139da0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x108502cd0>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x14e13a0c0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x108502cd0>), document_prompt=PromptTemplate(input_variables=['page_content'], templat

In [73]:
#定义模型
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)

In [74]:
#从hub中获取模版
from langchain import hub

# 一个最简单的模版,带记忆
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [75]:
#创建agent
from langchain.agents import create_openai_functions_agent
agent = create_openai_functions_agent(llm, tools, prompt)

In [76]:
#创建agent执行器AgentExecutor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [77]:
#执行
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [78]:
agent_executor.invoke({"input": "langsmith如何帮助做项目测试?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': '\nlangsmith project testing'}`


We provide native rendering of chat messages, functions, and retrieve documents.Initial Test Set‚ÄãWhile many developers still ship an initial version of their application based on ‚Äúvibe checks‚Äù, we‚Äôve seen an increasing number of engineering teams start to adopt a more test driven approach. LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications.
These test cases can be uploaded in bulk, created on the fly, or exported from application traces. LangSmith also makes it easy to run custom evaluations (both LLM and heuristic based) to score test results.Comparison View‚ÄãWhen prototyping different versions of your applications and making changes, it‚Äôs important to see whether or not you‚Äôve regressed with respect to your initial test cases.
Oftentimes, changes in 

{'input': 'langsmith如何帮助做项目测试?',
 'output': 'LangSmith can assist with project testing in several ways:\n\n1. **Creating Test Datasets**: Developers can create datasets that consist of collections of inputs and reference outputs. These datasets can be used to run tests on their LLM (Large Language Model) applications to ensure that the application behaves as expected.\n\n2. **Bulk Upload of Test Cases**: Test cases can be uploaded in bulk, which allows for efficient testing of multiple scenarios at once.\n\n3. **On-the-Fly Test Case Creation**: Developers can create test cases on the fly, which is useful for quickly testing new features or changes in the application.\n\n4. **Exporting from Application Traces**: Test cases can also be exported from application traces, which helps in creating tests based on real-world usage patterns.\n\n5. **Custom Evaluations**: LangSmith enables running custom evaluations, both LLM and heuristic-based, to score test results. This helps in assessing the

In [79]:
#实时信息提问
agent_executor.invoke({"input": "北京今天的天气如何?"})



> Entering new AgentExecutor chain...

Invoking: `search` with `{'query': '\nBeijing weather today'}`


{'type': 'weather_result', 'temperature': '53', 'unit': 'Fahrenheit', 'precipitation': '0%', 'humidity': '53%', 'wind': '5 mph', 'location': 'Beijing, China', 'date': 'Saturday', 'weather': 'Partly cloudy'}今天北京的天气是部分多云，气温为53华氏度（约11.7摄氏度），降水概率为0%，湿度为53%，风速为每小时5英里。

> Finished chain.


{'input': '北京今天的天气如何?',
 'output': '今天北京的天气是部分多云，气温为53华氏度（约11.7摄氏度），降水概率为0%，湿度为53%，风速为每小时5英里。'}

In [80]:
#前面的交互都不带记忆
agent_executor.invoke({"input": "截止目前我们都聊了什么?"})



> Entering new AgentExecutor chain...
到目前为止，我们还没有开始任何讨论。这是我们的第一次互动。如果您有任何问题或需要帮助，请随时告诉我，我会尽力协助您！

> Finished chain.


{'input': '截止目前我们都聊了什么?',
 'output': '到目前为止，我们还没有开始任何讨论。这是我们的第一次互动。如果您有任何问题或需要帮助，请随时告诉我，我会尽力协助您！'}

In [81]:
#交互时添加记忆
agent_executor.invoke({"input": "hi! 我叫bob", "chat_history": []})



> Entering new AgentExecutor chain...
你好，Bob！很高兴认识你。有什么我可以帮助你的吗？

> Finished chain.


{'input': 'hi! 我叫bob',
 'chat_history': [],
 'output': '你好，Bob！很高兴认识你。有什么我可以帮助你的吗？'}

In [82]:
#手动构造记忆数据
from langchain_core.messages import AIMessage, HumanMessage
agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="hi! 我叫bob"),
            AIMessage(content="你好，Bob！很高兴认识你。有什么我可以帮助你的吗？"),
        ],
        "input": "我叫什么名字?",
    }
)



> Entering new AgentExecutor chain...
你的名字是Bob。有什么我可以帮你的吗？

> Finished chain.


{'chat_history': [HumanMessage(content='hi! 我叫bob'),
  AIMessage(content='你好，Bob！很高兴认识你。有什么我可以帮助你的吗？')],
 'input': '我叫什么名字?',
 'output': '你的名字是Bob。有什么我可以帮你的吗？'}

In [83]:
#使用RunnableWithMessageHistory自动构造记忆数据
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
message_history = ChatMessageHistory()

In [84]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    #注意此处session_id没有用到，因为我们没用类似redis的存储,只是用了一个变量
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [85]:
#调用方式
agent_with_chat_history.invoke(
    {"input": "hi! 我是tomie"},
    #注意此处session_id没有用到，因为我们没用类似redis的存储,只是用了一个变量
    config={"configurable": {"session_id": "foo_001"}},
)



> Entering new AgentExecutor chain...
你好，Tomie！很高兴见到你。有什么我可以帮助你的吗？

> Finished chain.


{'input': 'hi! 我是tomie',
 'chat_history': [],
 'output': '你好，Tomie！很高兴见到你。有什么我可以帮助你的吗？'}

In [86]:
#调用方式
agent_with_chat_history.invoke(
    {"input": "我叫什么名字?"},
    #注意此处session_id没有用到，因为我们没用类似redis的存储,只是用了一个变量
    config={"configurable": {"session_id": "foo_001"}},
)



> Entering new AgentExecutor chain...
你刚才告诉我你的名字是Tomie。有什么我可以帮你的吗？

> Finished chain.


{'input': '我叫什么名字?',
 'chat_history': [HumanMessage(content='hi! 我是tomie'),
  AIMessage(content='你好，Tomie！很高兴见到你。有什么我可以帮助你的吗？')],
 'output': '你刚才告诉我你的名字是Tomie。有什么我可以帮你的吗？'}

In [87]:
#调用方式
agent_with_chat_history.invoke(
    {"input": "LangSmith如何使用?"},
    #注意此处session_id没有用到，因为我们没用类似redis的存储,只是用了一个变量
    config={"configurable": {"session_id": "foo_001"}},
)



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith \n'}`


LangSmith User Guide | ü¶úÔ∏èüõ†Ô∏è LangSmith

Skip to main contentü¶úÔ∏èüõ†Ô∏è LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they‚Äôre just starting their journey.Prototyping‚ÄãPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.

We provide native rendering of chat messages, funct

{'input': 'LangSmith如何使用?',
 'chat_history': [HumanMessage(content='hi! 我是tomie'),
  AIMessage(content='你好，Tomie！很高兴见到你。有什么我可以帮助你的吗？'),
  HumanMessage(content='我叫什么名字?'),
  AIMessage(content='你刚才告诉我你的名字是Tomie。有什么我可以帮你的吗？')],
 'output': 'LangSmith是一个用于LLM（大型语言模型）应用开发、监控和测试的平台。以下是如何使用LangSmith的一些基本指南：\n\n1. **原型设计**：在LLM应用的原型设计阶段，通常涉及到快速在不同的提示（prompts）、模型类型、检索策略和其他参数之间进行实验。LangSmith提供了聊天消息、函数和检索文档的原生渲染。\n\n2. **初始测试集**：开发者可以创建数据集，这些数据集是输入和参考输出的集合，并使用这些数据集在LLM应用上运行测试。测试用例可以批量上传、即时创建或从应用跟踪中导出。LangSmith还可以轻松运行自定义评估（基于LLM和启发式的）来评分测试结果。\n\n3. **比较视图**：在原型设计不同版本的应用并进行更改时，重要的是要看到你是否在初始测试用例方面有所退步。提示、检索策略或模型选择的变化可能对应用程序产生的响应有很大影响。快速理解模型的性能并调试失败的地方对于这个阶段非常重要。\n\n4. **调试**：在开发新的LLM应用时，建议默认启用LangSmith跟踪。通常不需要查看每一个跟踪，但当出现问题（意外的结果、无限代理循环、执行速度慢于预期、令牌使用量高于预期）时，通过查看应用跟踪来调试非常有帮助。LangSmith在LLM序列的每一步提供清晰的可见性和调试信息，使得识别和找出问题根源变得更加容易。\n\n这些是LangSmith的一些基本使用指南。如果你有更具体的问题或需要进一步的帮助，请告诉我！'}

In [88]:
#调用方式
agent_with_chat_history.invoke(
    {"input": "截止目前我们都聊了什么?"},
    #注意此处session_id没有用到，因为我们没用类似redis的存储,只是用了一个变量
    config={"configurable": {"session_id": "foo_001"}},
)



> Entering new AgentExecutor chain...
到目前为止，我们的对话内容如下：

1. 你向我打招呼，并告诉我你的名字是Tomie。
2. 我问你是否需要帮助。
3. 你询问了你的名字，我确认了你之前告诉我的名字是Tomie。
4. 你问了关于LangSmith如何使用的问题，我给你提供了关于LangSmith使用的一些基本指南。

如果你有其他问题或需要进一步的帮助，请随时告诉我！

> Finished chain.


{'input': '截止目前我们都聊了什么?',
 'chat_history': [HumanMessage(content='hi! 我是tomie'),
  AIMessage(content='你好，Tomie！很高兴见到你。有什么我可以帮助你的吗？'),
  HumanMessage(content='我叫什么名字?'),
  AIMessage(content='你刚才告诉我你的名字是Tomie。有什么我可以帮你的吗？'),
  HumanMessage(content='LangSmith如何使用?'),
  AIMessage(content='LangSmith是一个用于LLM（大型语言模型）应用开发、监控和测试的平台。以下是如何使用LangSmith的一些基本指南：\n\n1. **原型设计**：在LLM应用的原型设计阶段，通常涉及到快速在不同的提示（prompts）、模型类型、检索策略和其他参数之间进行实验。LangSmith提供了聊天消息、函数和检索文档的原生渲染。\n\n2. **初始测试集**：开发者可以创建数据集，这些数据集是输入和参考输出的集合，并使用这些数据集在LLM应用上运行测试。测试用例可以批量上传、即时创建或从应用跟踪中导出。LangSmith还可以轻松运行自定义评估（基于LLM和启发式的）来评分测试结果。\n\n3. **比较视图**：在原型设计不同版本的应用并进行更改时，重要的是要看到你是否在初始测试用例方面有所退步。提示、检索策略或模型选择的变化可能对应用程序产生的响应有很大影响。快速理解模型的性能并调试失败的地方对于这个阶段非常重要。\n\n4. **调试**：在开发新的LLM应用时，建议默认启用LangSmith跟踪。通常不需要查看每一个跟踪，但当出现问题（意外的结果、无限代理循环、执行速度慢于预期、令牌使用量高于预期）时，通过查看应用跟踪来调试非常有帮助。LangSmith在LLM序列的每一步提供清晰的可见性和调试信息，使得识别和找出问题根源变得更加容易。\n\n这些是LangSmith的一些基本使用指南。如果你有更具体的问题或需要进一步的帮助，请告诉我！')],
 'output': '到目前为止，我们的对话内容如下：\n\n1. 你向我打招呼，并告诉我你的名字是Tomie。\n2. 我问你是否需要帮助。\n3